# Batch IK convergence analysis
This notebook analysis the emprirical convergence properties of the two batch ik methods - the jacobian psuedo inverse method and the auto-diff method 

In [ ]:
%load_ext autoreload
%autoreload 2
from IPython.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))

In [ ]:
from time import time
from typing import Callable
from dataclasses import dataclass

import numpy as np
import torch
import pandas as pd
import matplotlib.pyplot as plt

from jrl.utils import to_torch, set_seed
from jrl.robot import Robot
from jrl.robots import Panda
from jrl.conversions import geodesic_distance_between_quaternions
from jrl.evaluation import solution_pose_errors

set_seed(0)
assert torch.cuda.is_available()

In [ ]:
def fn_mean_std(fn: Callable, k: int):
    runtimes = []
    for _ in range(k):
        t0 = time()
        fn()
        runtimes.append(1000 * (time() - t0))
    return np.mean(runtimes), np.std(runtimes)

@dataclass
class OptimStepEval:
    name: str
    mean_t_err: float
    mean_R_err: float
    t_err_std: float
    R_err_std: float
    alpha: float
    t_elapsed: float

In [ ]:
robot = Panda()

In [ ]:
# def klampt

methods = {
    # "Klampt invJac cpu": lambda: robot.inverse_kinematics_single_step_batch_pt(goalposes_cpu, x_pt_cpu),
    "jac-pinv (J from klampt)": lambda x, target, alpha: robot.inverse_kinematics_single_step_batch_pt(target, x, alpha=alpha),
    "jac-pinv (J from klampt)": lambda x, target, alpha: robot.inverse_kinematics_single_step_batch_pt(target, x, alpha=alpha),
    "AutoDiff": lambda x, target, alpha: robot.inverse_kinematics_autodiff_single_step_batch_pt(target, x, alpha=alpha),
    "Levenburg-Marquardt": lambda x, target, alpha: robot.inverse_kinematics_single_step_levenburg_marquardt(target, x),
}

batch_size = 10

goalpose_angles, goalposes = robot.sample_joint_angles_and_poses(batch_size)
goalposes_cuda = to_torch(goalposes.copy(), device="cuda")
# x_pt, _ = robot.sample_joint_angles_and_poses(batch_size) # randomly drawn
# x_pt = to_torch(x_pt).cuda()
x_pt = to_torch(goalpose_angles.copy()).cuda() # close to solution
x_pt = robot.clamp_to_joint_limits(x_pt + torch.randn_like(x_pt) / 10)
    
df = pd.DataFrame(
    columns=[
        "method", 
        "alpha", 
        "number of solutions", 
        "total runtime (s)", 
        "number of optimization steps", 
        "final mean translational error (cm)", 
        "final mean rotational error (deg)"
    ])

all_loss_histories = []
for name, method in methods.items():
    
#     for alpha in [0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
#     for alpha in [0.01, 0.1, 0.2, 0.3, 0.4, 0.5]:
    for alpha in [0.2]:
        
        loss_history = []
        x_pt_i = x_pt.detach()

        t_elapsed = 0
        counter = 0

        while (len(loss_history) == 0 or loss_history[-1].mean_t_err > 0.1) and counter < 200:
            l2_errors, ang_errors = solution_pose_errors(robot, x_pt_i, goalposes_cuda)
            l2_errors = 100 * l2_errors
            ang_errors = np.rad2deg(ang_errors)

            loss_history.append(
                OptimStepEval(
                    name=name,
                    mean_t_err=l2_errors.mean().item(),
                    mean_R_err=ang_errors.mean().item(),
                    t_err_std=l2_errors.mean().item(),
                    R_err_std=ang_errors.mean().item(),
                    alpha=alpha,
                    t_elapsed=t_elapsed))

            t0i = time()
            x_pt_i = method(x_pt_i, goalposes_cuda, alpha)
            t_elapsed += time() - t0i
            counter += 1 
            
        new_row = [name, alpha, batch_size, t_elapsed, counter, loss_history[-1].mean_t_err, loss_history[-1].mean_R_err]
        df.loc[len(df)] = new_row
        all_loss_histories.append(loss_history)
        
        if "Levenburg" in name: 
            break

        
df = df.sort_values(by=["method", "alpha"])
df_success = df[df['final mean translational error (cm)'] < 0.1]
df_success.sort_values(by=["total runtime (s)"])

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15, 6))
fig.suptitle(f"IK convergence of configs close to solutions - batch_size={batch_size}")

TARGET_t_err = 0.1
max_n_steps_plotted = 60


for loss_history in all_loss_histories:
    tsteps = np.arange(len(loss_history))    
    telapsed = np.array([ose.t_elapsed for ose in loss_history])
    mean_ts = np.array([ose.mean_t_err for ose in loss_history])
    std_ts = np.array([ose.t_err_std for ose in loss_history])
    
    if len(loss_history) >= max_n_steps_plotted:
        tsteps = tsteps[0:max_n_steps_plotted]
        telapsed = telapsed[0:max_n_steps_plotted]
        mean_ts = mean_ts[0:max_n_steps_plotted]
        std_ts = std_ts[0:max_n_steps_plotted]
        
    label = f"{loss_history[0].name}, alpha={loss_history[0].alpha}"

    axs[0].plot(tsteps, TARGET_t_err*np.ones(tsteps.size), color="green", linestyle="dotted")
    axs[1].plot(telapsed, TARGET_t_err*np.ones(tsteps.size), color="green", linestyle="dotted")
    
    if mean_ts[-1] < TARGET_t_err:
        axs[0].plot(tsteps, mean_ts, label=label)
        axs[1].plot(telapsed, mean_ts, label=label)
    else:
        axs[0].plot(tsteps, mean_ts, color="red", alpha=0.15, label=label)
        axs[1].plot(telapsed, mean_ts, color="red", alpha=0.15, label=label)
    
for ax in axs:
    ax.set_ylabel("Mean translational error (cm)")
    ax.set_ylim(0, 10)

axs[1].legend()
axs[0].set_xlabel("Steps")
axs[1].set_xlabel("Runtime (s)")